In [1]:
from dataclasses import dataclass
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool
from langgraph.runtime import get_runtime

USER_DATABASE = {
    "user123": {
        "name": "Alice Johnson",
        "account_type": "Premium",
        "balance": 5000,
        "email": "alice@example.com"
    },
    "user456": {
        "name": "Bob Smith",
        "account_type": "Standard",
        "balance": 1200,
        "email": "bob@example.com"
    }
}

@dataclass
class UserContext:
    user_id: str

@tool
def get_account_info() -> str:
    """Get the current user's account information."""
    runtime = get_runtime(UserContext)
    user_id = runtime.context.user_id

    if user_id in USER_DATABASE:
        user = USER_DATABASE[user_id]
        return f"Account holder: {user['name']}\nType: {user['account_type']}\nBalance: ${user['balance']}"
    return "User not found"

model = ChatOpenAI(
    base_url="http://localhost:12434/engines/v1",
    api_key="docker",
    temperature=0,
    model="ai/gemma3"
)
agent = create_agent(
    model,
    tools=[get_account_info],
    context_schema=UserContext,
    system_prompt="You are a financial assistant."
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my current balance?"}]},
    context=UserContext(user_id="user123")
)

In [2]:
result

{'messages': [HumanMessage(content="What's my current balance?", additional_kwargs={}, response_metadata={}, id='a8d515e3-a48f-4cf2-af6c-9fc6b5ac2f77'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 227, 'total_tokens': 264, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ai/gemma3', 'system_fingerprint': 'b1-ca71fb9', 'id': 'chatcmpl-d1i3jlVuCLjRrRb9pXIbCxuT8MvdUMlQ', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--9aecb027-474c-4ad7-a164-a71d03ea4371-0', tool_calls=[{'name': 'get_account_info', 'args': {}, 'id': 'HbYG4SvEFfVbc4EgzqcIv2pO7OKSdyX5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 227, 'output_tokens': 37, 'total_tokens': 264, 'input_token_details': {}, 'output_token_details': {}}),
  ToolMessage(content='Account holder: Alice Johnson\nType: Premium\nBalance: $5000', name='get

In [3]:
# Memory
from typing import Any
from langgraph.runtime import get_runtime
from langgraph.store.memory import InMemoryStore
from langchain.agents import create_agent
from langchain.tools import tool

# Access memory
@tool
def get_user_info(user_id: str) -> str:
    """Look up user info."""
    store = get_runtime().store
    user_info = store.get(("users",), user_id)
    return str(user_info.value) if user_info else "Unknown user"

# Update memory
@tool
def save_user_info(user_id: str, user_info: dict[str, Any]) -> str:
    """Save user info."""
    store = get_runtime().store
    store.put(("users",), user_id, user_info)
    return "Successfully saved user info."

store = InMemoryStore()
agent = create_agent(
    model,
    tools=[get_user_info, save_user_info],
    store=store
)

# First session: save user info
agent.invoke({
    "messages": [{"role": "user", "content": "Save the following user: userid: abc123, name: Foo, age: 25, email: foo@langchain.dev"}]
})

# Second session: get user info
agent.invoke({
    "messages": [{"role": "user", "content": "Get user info for user with id 'abc123'"}]
})
# Here is the user info for user with ID "abc123":
# - Name: Foo
# - Age: 25
# - Email: foo@langchain.dev

{'messages': [HumanMessage(content="Get user info for user with id 'abc123'", additional_kwargs={}, response_metadata={}, id='dfa55e31-ee53-4311-9875-7837ff477b95'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 395, 'total_tokens': 445, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ai/gemma3', 'system_fingerprint': 'b1-ca71fb9', 'id': 'chatcmpl-tyxUnGgEjiJUcSW6jPnWBVY4nZD6YDuB', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--e5692d6e-d12e-48fb-bd9e-f7ead9d9b873-0', tool_calls=[{'name': 'get_user_info', 'args': {'user_id': 'abc123'}, 'id': '7hyeDkr7qPu1ARtQfGMlJXEpBCCZjbQh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 395, 'output_tokens': 50, 'total_tokens': 445, 'input_token_details': {}, 'output_token_details': {}}),
  ToolMessage(content="{'name': 'Foo', 'age': 25, 'email': 'foo@la